# Spark

TODO:
* union all jsons together
    * see how much it is on the mem?
* get the desired metrics
* (?,*) push to Clickhouse for storage

## Implementation


In [1]:
import pathlib, os

In [2]:
spark_resources_path = "/home/jovyan/work/data/"

In [3]:
# def gharchive_path(dt):
#     return "{:04d}-{:02d}-{:02d}-{}.json.gz".format(dt.year, dt.month, dt.day, dt.hour)

In [4]:
spark_resources_path_obj = pathlib.Path(spark_resources_path)

In [5]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
paths = [str(path) for path in sorted(spark_resources_path_obj.glob("*.json.gz"), key=os.path.getmtime)]
paths

['/home/jovyan/work/data/2023-07-27-0.json.gz',
 '/home/jovyan/work/data/2023-07-26-23.json.gz',
 '/home/jovyan/work/data/2023-07-27-2.json.gz',
 '/home/jovyan/work/data/2023-07-27-4.json.gz',
 '/home/jovyan/work/data/2023-07-27-8.json.gz',
 '/home/jovyan/work/data/2023-07-27-5.json.gz',
 '/home/jovyan/work/data/2023-07-27-6.json.gz',
 '/home/jovyan/work/data/2023-07-27-3.json.gz',
 '/home/jovyan/work/data/2023-07-27-1.json.gz',
 '/home/jovyan/work/data/2023-07-27-7.json.gz',
 '/home/jovyan/work/data/2023-07-27-9.json.gz',
 '/home/jovyan/work/data/2023-07-27-10.json.gz',
 '/home/jovyan/work/data/2023-07-27-11.json.gz',
 '/home/jovyan/work/data/2023-07-27-12.json.gz',
 '/home/jovyan/work/data/2023-07-27-13.json.gz',
 '/home/jovyan/work/data/2023-07-27-14.json.gz',
 '/home/jovyan/work/data/2023-07-27-15.json.gz',
 '/home/jovyan/work/data/2023-07-27-16.json.gz',
 '/home/jovyan/work/data/2023-07-27-17.json.gz',
 '/home/jovyan/work/data/2023-07-27-18.json.gz',
 '/home/jovyan/work/data/2023-

In [17]:
paths[-10:]

['/home/jovyan/work/data/2023-07-27-18.json.gz',
 '/home/jovyan/work/data/2023-07-27-19.json.gz',
 '/home/jovyan/work/data/2023-07-27-20.json.gz',
 '/home/jovyan/work/data/2023-07-27-21.json.gz',
 '/home/jovyan/work/data/2023-07-27-22.json.gz',
 '/home/jovyan/work/data/2023-07-27-23.json.gz',
 '/home/jovyan/work/data/2023-07-28-0.json.gz',
 '/home/jovyan/work/data/2023-07-28-1.json.gz',
 '/home/jovyan/work/data/2023-07-28-2.json.gz',
 '/home/jovyan/work/data/2023-07-28-3.json.gz']

In [18]:
df = spark.read.json(paths[-10:])

In [ ]:
# paths = spark_resources_path_obj.glob("*.json.gz")
# tpath = str(next(paths))
# print(tpath)
# df = spark.read.json(tpath)
    

In [19]:
df.show()

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|             type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|{https://avatars....|2023-07-27T18:00:00Z|30723864809|                null|{started, null, n...|  true|{643819103, AbidH...|       WatchEvent|
|{https://avatars....|2023-07-27T18:00:00Z|30723864812|                null|{null, 8d8a695447...|  true|{670881342, moyan...|        PushEvent|
|{https://avatars....|2023-07-27T18:00:00Z|30723864821|                null|{null, a380680778...|  true|{603317401, token...|        PushEvent|
|{https://avatars....|2023-07-27T18:00:00Z|30723864847|                null|{null, 3202b09329...|  true|{671166464, TheCa...|        Pus

In [20]:
df.count()

1479561

In [21]:
df.createOrReplaceTempView("activity")

In [22]:
spark.sql(
    "SELECT min(created_at), max(created_at) FROM activity"
).show(10)

+--------------------+--------------------+
|     min(created_at)|     max(created_at)|
+--------------------+--------------------+
|2023-07-27T18:00:00Z|2023-07-28T03:59:59Z|
+--------------------+--------------------+



In [23]:
spark.sql(
    "SELECT hour(created_at) as hour, count(*) FROM activity group by hour"
).show(10)

+----+--------+
|hour|count(1)|
+----+--------+
|  18|  181467|
|  19|  166557|
|  20|  161624|
|  21|  150205|
|  22|  132519|
|   0|  146356|
|   2|  137783|
|   1|  150680|
|  23|  125156|
|   3|  127214|
+----+--------+



In [26]:
spark.sql(
    "SELECT type, count(*) FROM activity group by type order by count(*) desc"
).show(50)

+--------------------+--------+
|                type|count(1)|
+--------------------+--------+
|           PushEvent|  878497|
|         CreateEvent|  179214|
|    PullRequestEvent|  101843|
|          WatchEvent|   70209|
|   IssueCommentEvent|   68660|
|         DeleteEvent|   44482|
|PullRequestReview...|   38227|
|         IssuesEvent|   30602|
|PullRequestReview...|   23592|
|           ForkEvent|   17069|
|        ReleaseEvent|    8627|
|  CommitCommentEvent|    7350|
|         PublicEvent|    4931|
|         MemberEvent|    3625|
|         GollumEvent|    2633|
+--------------------+--------+



## Metrics

* List of Developers that own more than one repository;
* List of Developers who did more than one commit in a day, ordered by name and number of commits;
* List of Developers with less than one commit in a day;
* Total Developers grouped by gender;
* Total projects with more than 10 members;

### LoD - Own > 1 Repo

In [40]:
df.select(
    df["actor.display_login"],
    df["actor.id"],
    df["actor.login"],
    df["repo.id"],
    df["repo.name"],
    df["type"]
).show(20)

+----------------+---------+-------------------+---------+--------------------+-----------------+
|   display_login|       id|              login|       id|                name|             type|
+----------------+---------+-------------------+---------+--------------------+-----------------+
|   HippomasAKiB1|132694624|      HippomasAKiB1|643819103|AbidHasanPiash/bh...|       WatchEvent|
|        moyan222|140571676|           moyan222|670881342|moyan222/moyan222...|        PushEvent|
|       tokenhash|100255150|          tokenhash|603317401|tokenhash/twitter...|        PushEvent|
|     TheCamBloch|129989249|        TheCamBloch|671166464|TheCamBloch/Jetso...|        PushEvent|
|     peterpolman|  2378695|        peterpolman|514247259|thxprotocol/monorepo|        PushEvent|
|       wlosantos| 21031611|          wlosantos|671540904|  wlosantos/rails-CI|        PushEvent|
|           uplau|139003180|              uplau|671603706|uplau/rust-templa...|      CreateEvent|
|         wrygiel|  

In [42]:
split_col = pyspark.sql.functions.split(df['repo.name'], '/')

df = df.withColumn('repo_author', split_col.getItem(0))
df = df.withColumn('repo_name', split_col.getItem(1))

In [44]:
df.createOrReplaceTempView("activity")

In [48]:
spark.sql(
    """
    SELECT 
        repo_author, 
        count(repo_name) as total_repo_events, 
        count(distinct repo_name) as total_repos 
    FROM activity 
    GROUP BY repo_author
    HAVING total_repos > 1
    ORDER BY total_repos DESC
    """
).show(50)

+--------------------+-----------------+-----------+
|         repo_author|total_repo_events|total_repos|
+--------------------+-----------------+-----------+
|     direwolf-github|             7643|       1163|
|      pmacik-testing|             2724|        553|
| learn-co-curriculum|              748|        473|
|           microsoft|             5106|        416|
| redhat-appstudio-qe|             1901|        281|
|tencentcloud-sdk-php|              471|        236|
|              google|             1254|        207|
|              apache|             2541|        187|
|          tf-vcs-e2e|              512|        174|
|    msft-mirror-aosp|              295|        172|
|                Mu-L|              475|        170|
|                 KDE|              323|        159|
|         conda-forge|              977|        155|
|             kkpan11|              456|        154|
|        repo-rangers|              150|        150|
|              sarvex|              670|      

In [49]:
spark.sql(
    """
    SELECT count(distinct sq.repo_author) from 
    (
        SELECT 
            repo_author, 
            count(repo_name) as total_repo_events, 
            count(distinct repo_name) as total_repos 
        FROM activity 
        GROUP BY repo_author
        HAVING total_repos > 1
        ORDER BY total_repos DESC
    ) sq
    """
).show(50)

+---------------------------+
|count(DISTINCT repo_author)|
+---------------------------+
|                      38046|
+---------------------------+



#### Limitations & Problems

1. As the snapshots in GHARCHIVE are hourly-based, there is __no data__ about the past activity for the repositories. \
So without the past activity, the number of owned repos is incomplete and can be based only on the activities, that we've crawled & stored. \
On how to overcome this, please read the next section, `How can it be improved?`

#### How can it be improved?

* Get bigger slice of data - and do the next thing:
  * Store daily snapshots of all unique combinations of `date`, `repo.author`, `repo.name`
  * Provide the metrics by getting the global `count(distinct repo.name)` aggregation on group of `repo.author`-s

### LoD - Did > 1 Commit a day

https://stackoverflow.com/questions/64605066/explode-array-with-nested-array-raw-spark-sql

#### Notes

1. Commits are present inside `payload.commits.author.name`.\
Given the hourly and typized nature of the dataset, we can:
    * Filter by `type`, leaving only `PushEvent`-s
    * Explode author names
    * Get the count aggregation

#### Impl

In [96]:
df.select(
    df["payload.commits.author.name"],
    df["payload.ref"],
    df["payload.ref_type"],
    df["payload.review"],
    df["type"],
).show(50)

+--------------------+--------------------+----------+--------------------+--------------------+
|                name|                 ref|  ref_type|              review|                type|
+--------------------+--------------------+----------+--------------------+--------------------+
|                null|                null|      null|                null|          WatchEvent|
|          [moyan222]|     refs/heads/main|      null|                null|           PushEvent|
|         [tokenhash]|     refs/heads/main|      null|                null|           PushEvent|
|       [TheCamBloch]|     refs/heads/main|      null|                null|           PushEvent|
|             [Peter]|     refs/heads/safe|      null|                null|           PushEvent|
|       [wendellopes]|refs/heads/featur...|      null|                null|           PushEvent|
|                null|     lib-default-use|       tag|                null|         CreateEvent|
|[DEV Registry Ser...|   refs/

In [71]:
spark.sql(
    """
    SELECT
        date(created_at) as created_at_date,
        explode(payload.commits.author.name) as author_name
    FROM activity
    WHERE 
        type = 'PushEvent'
    """
).show(10)

+---------------+--------------------+
|created_at_date|         author_name|
+---------------+--------------------+
|     2023-07-27|            moyan222|
|     2023-07-27|           tokenhash|
|     2023-07-27|         TheCamBloch|
|     2023-07-27|               Peter|
|     2023-07-27|         wendellopes|
|     2023-07-27|DEV Registry Service|
|     2023-07-27|DEV Registry Service|
|     2023-07-27|             preciad|
|     2023-07-27| github-actions[bot]|
|     2023-07-27| github-actions[bot]|
+---------------+--------------------+
only showing top 10 rows



In [72]:
spark.sql(
    """
    SELECT 
        sq.created_at_date as date,
        sq.author_name, 
        count(*) total_commits from 
    (
        SELECT
            date(created_at) as created_at_date,
            explode(payload.commits.author.name) as author_name
        FROM activity
        WHERE 
            type = 'PushEvent'
    ) sq
    GROUP BY 
        date, 
        sq.author_name
    HAVING total_commits > 1
    ORDER BY 
        total_commits DESC, 
        author_name DESC
    """
).show(10)

+----------+-------------------+-------------+
|      date|        author_name|total_commits|
+----------+-------------------+-------------+
|2023-07-28|github-actions[bot]|        64469|
|2023-07-27|github-actions[bot]|        35880|
|2023-07-27|        Upptime Bot|        27881|
|2023-07-28|        Upptime Bot|        13536|
|2023-07-27|      renovate[bot]|        10954|
|2023-07-27|           sgou1969|         8488|
|2023-07-27|    dependabot[bot]|         8071|
|2023-07-28|           sgou1969|         6061|
|2023-07-28|      renovate[bot]|         5313|
|2023-07-27|         readme-bot|         5125|
+----------+-------------------+-------------+
only showing top 10 rows



#### Limitations & Problems

1. Potentially, subquery usage in Spark is not a good practice. Yet, it may be used for prototype reasons.
2. We have to additionally group by date, because we have to provide DAILY number of commits.

#### How can it be improved?

* Probably, we should get rid of bot entities :)

### [!!!] LoD - < 1 commit in a day

#### Notes

* The trick is that an author can have 1 commit in, let's say, 25h time span, which is more than 1 day. \
So we should regroup data not by a date, but by consecutive dataspansm which are 25h each...
* For to test it, we need at least __2 days worth of data__! \
For now I have complications to get this data, so I skip the metric.

#### Impl

In [73]:
spark.sql(
    """
    SELECT 
        sq.created_at_date as date,
        sq.author_name, 
        count(*) total_commits from 
    (
        SELECT
            date(created_at) as created_at_date,
            explode(payload.commits.author.name) as author_name
        FROM activity
        WHERE 
            type = 'PushEvent'
    ) sq
    GROUP BY 
        date, 
        sq.author_name
    HAVING total_commits <= 1
    ORDER BY 
        total_commits DESC, 
        author_name DESC
    """
).show(10)

+----------+--------------------+-------------+
|      date|         author_name|total_commits|
+----------+--------------------+-------------+
|2023-07-28|            🤖 R2-D2|            1|
|2023-07-28|      💵 moneybot 💵|            1|
|2023-07-28| 🐼 Samrose Ahmed 🐼|            1|
|2023-07-27|𒀳 Scribe of the ...|            1|
|2023-07-27|                  ｚ|            1|
|2023-07-28|              황지환|            1|
|2023-07-28|              홍세빈|            1|
|2023-07-27|              현기홍|            1|
|2023-07-28|                현경|            1|
|2023-07-27|              한창수|            1|
+----------+--------------------+-------------+
only showing top 10 rows



### [!!!] Total Developers grouped by gender

#### Notes

* Proabably can be done via `user[profile_pronouns]` , but for EACH unique GitHub User... That's just dumb.
* I haven't found such info (sex,gender, pronouns) in the dataset.

### Total projects with more than 10 members

#### Notes

* Where to find members ???

#### IMPL


In [98]:
spark.sql(
    """
    SELECT
        type,
        payload.member
    FROM activity
    WHERE
        payload.member is not NULL
        
    """
).show(10)

+-----------+--------------------+
|       type|              member|
+-----------+--------------------+
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
|MemberEvent|{https://avatars....|
+-----------+--------------------+
only showing top 10 rows



In [114]:
df.select(
    df["payload.member"],
    df["payload.member.id"],
    df["payload.member.login"],
    df["payload.member.type"],
    df["payload.action"],
    df["org.login"],
    df["actor.login"],
    df["repo.name"],
    df["type"],
).filter(df["type"] == 'MemberEvent').show(50)

+--------------------+---------+-------------------+----+------+--------------------+--------------------+--------------------+-----------+
|              member|       id|              login|type|action|               login|               login|                name|       type|
+--------------------+---------+-------------------+----+------+--------------------+--------------------+--------------------+-----------+
|{https://avatars....|107618870|            bsozeau|User| added|                null|      noirblancrouge|noirblancrouge/Pa...|MemberEvent|
|{https://avatars....|132350039|         RolivhuwaN|User| added|                null|        Spinofficial| Spinofficial/printf|MemberEvent|
|{https://avatars....|132908419|        stheeCamile|User| added|                null|              faellm| faellm/calculoRotas|MemberEvent|
|{https://avatars....|140736057|       bevin-crypto|User| added|                null|       Programmer231|Programmer231/Roc...|MemberEvent|
|{https://avatars...

In [118]:
spark.sql(
    """
    SELECT
        repo.name,
        count(distinct payload.member.login) as total_member_logins
    FROM activity
    WHERE
        type = 'MemberEvent'
        and payload.action = 'added'
    GROUP BY
        repo.name
    ORDER BY
        total_member_logins DESC
    """
).show(10)

+--------------------+-------------------+
|                name|total_member_logins|
+--------------------+-------------------+
|jgranadoscunoc/re...|                 29|
|Jucer74/WebDevelo...|                 17|
|hdtoledo/nodejs_a...|                 11|
|InstrucJavaReclui...|                  9|
|ChungLeba/nextjsv...|                  9|
|iramgutierrez/433...|                  9|
|emrchi/TestNGProj...|                  8|
|aydaakcay/TestPro...|                  7|
|ufuk-muhsiroglu/T...|                  7|
|GoldenMEmre/com.w...|                  7|
+--------------------+-------------------+
only showing top 10 rows



In [119]:
spark.sql(
    """
    SELECT
        date(created_at) as date,
        hour(created_at) as hour,
        repo.name,
        count(distinct payload.member.login) as total_member_logins
    FROM activity
    WHERE
        type = 'MemberEvent'
        and payload.action = 'added'
    GROUP BY
        date, hour, repo.name
    ORDER BY
        total_member_logins DESC
    """
).show(10)

+----------+----+--------------------+-------------------+
|      date|hour|                name|total_member_logins|
+----------+----+--------------------+-------------------+
|2023-07-27|  20|jgranadoscunoc/re...|                 21|
|2023-07-28|   1|Jucer74/WebDevelo...|                 16|
|2023-07-28|   1|InstrucJavaReclui...|                  9|
|2023-07-27|  19|hdtoledo/nodejs_a...|                  9|
|2023-07-27|  19|emrchi/TestNGProj...|                  7|
|2023-07-27|  19|ufuk-muhsiroglu/T...|                  7|
|2023-07-27|  18|aydaakcay/TestPro...|                  7|
|2023-07-27|  18|GoldenMEmre/com.w...|                  7|
|2023-07-27|  19|mehmetfilik/comWo...|                  6|
|2023-07-27|  19|FabianoCarneiro/p...|                  6|
+----------+----+--------------------+-------------------+
only showing top 10 rows



In [122]:
spark.sql(
    """
    SELECT
        date(created_at) as date,
        hour(created_at) as hour,
        created_at,
        repo.name,
        payload.member.login as member_name
    FROM activity
    WHERE
        type = 'MemberEvent'
        and payload.action = 'added'
    ORDER BY
        date ASC,
        hour ASC,
        created_at ASC,
        repo.name ASC,
        member_name ASC
    """
).show(10)

+----------+----+--------------------+--------------------+---------------+
|      date|hour|          created_at|                name|    member_name|
+----------+----+--------------------+--------------------+---------------+
|2023-07-27|  18|2023-07-27T18:00:15Z|noirblancrouge/Pa...|        bsozeau|
|2023-07-27|  18|2023-07-27T18:00:31Z| Spinofficial/printf|     RolivhuwaN|
|2023-07-27|  18|2023-07-27T18:00:33Z| faellm/calculoRotas|    stheeCamile|
|2023-07-27|  18|2023-07-27T18:00:39Z|Programmer231/Roc...|   bevin-crypto|
|2023-07-27|  18|2023-07-27T18:00:41Z|    Drefdu/DentalNew|AntonioEstrada0|
|2023-07-27|  18|2023-07-27T18:00:44Z|code-X16/simple_s...|      udeme-goc|
|2023-07-27|  18|2023-07-27T18:01:13Z|maxDes23/project-...|       MadDog83|
|2023-07-27|  18|2023-07-27T18:01:15Z|Diego582/ideas-pi...|       FedeSabi|
|2023-07-27|  18|2023-07-27T18:01:16Z|CaravanStudios/dc...| caseylivingood|
|2023-07-27|  18|2023-07-27T18:01:20Z|devfabien/Gym-git...| IbrahimBagalwa|
+----------+

#### Limitations & Problems

1. As the snapshots in GHARCHIVE are hourly-based, there is __no data__ about the past activity for the repositories. \
So without the past activity, the number of members is incomplete and can be based only on the activities, like 'MemberEvent', that we've crawled & stored. \
On how to overcome this, please read the next section, `How can it be improved?`

#### How can it be improved?

* Get bigger slice of data - and do the next thing:
  * Store daily snapshots of all unique combinations of `date`, `repo`, `member`, 'action'
  * Provide the metrics by getting the global `count(distinct member)` aggregation on group of `repo`-s